## 下載檔案

In [1]:
import re 
import pandas as pd 
import pdfplumber as pr
import requests
# 設置 PDF 檔案的下載鏈接和檔案名稱
company_code = "2883"
year = "110"

url = "https://mops.twse.com.tw/server-java/FileDownLoad?step=9&filePath=/home/html/nas/protect/t100/&fileName=t100sa11_" + company_code + "_" + year + ".pdf"
filename = "t100sa11_" + company_code + "_" + year + ".pdf"

# 發送 GET 請求並下載檔案
response = requests.get(url)
with open(filename, "wb") as f:
    f.write(response.content)


## 讀取GRI表格

In [2]:
pdf = pr.open(filename)
ps = pdf.pages
df_new=[]
 # 取得總頁數
total_pages = len(pdf.pages)
# 設定起始頁數
start_page = int(total_pages * 0.8)
    
# 迭代每頁，從設定的起始頁開始
for page in pdf.pages[start_page:]:
    tables = page.extract_tables()
    for df in tables:
        df = pd.DataFrame(df, columns=df[0])
        if df.iloc[0,0] != '': 
            #df=df.drop_duplicates()
            for i in df.columns:
                filtered_columns = [col for col in df.columns if col is not None]
                for col in filtered_columns:
                    if  ("GRI" ) in col:
                            df_new.append(df) 

                

In [10]:
df_new[2]

,GRI\n準則,揭露項目,None,None
0,GRI\n準則,揭露項目,None,None
1,一般揭露,None,章節,頁數
2,None,102-1\n組織的名稱,關於本報告書,3
3,None,102-2\n活動、品牌、產品與服務,企業概況,6
4,None,102-3\n總部位置,企業概況,6
5,None,102-4\n營運據點,企業概況,6
6,None,102-5\n所有權與法律形式,企業概況,6
7,None,102-6\n提供服務的市場,企業概況,6
8,None,102-7\n組織規模,企業概況,6
9,None,102-8\n員工與其他工作者的資訊,4.1\n員工結構,95


## 合併表格

In [11]:
merged_df = pd.concat(df_new, axis=0)
merged_df

,GRI\n準則,揭露項目,None,None
0,GRI\n準則,揭露項目,None,None
1,一般揭露,None,章節,頁數
2,None,102-1\n組織的名稱,關於本報告書,3
3,None,102-2\n活動、品牌、產品與服務,企業概況,6
4,None,102-3\n總部位置,企業概況,6
...,...,...,...,...
12,None,308-2\n供應鏈對環境的負面衝擊，以及所採取的行動,5.2.1\n供應商管理,121
13,GRI 408 2016\n：童工,408-1\n營運據點和供應商使用童工之重大風險,5.2.1\n供應商管理,121
14,GRI 409 2016\n：強迫或強制勞動,409-1\n具強迫或強制勞動事件重大風險的營運據點和供應商,5.2.1\n供應商管理,121
15,None,414-1\n使用社會準則篩選之新供應商,5.2.1\n供應商管理,121


## 另一種擷取方式，先判斷哪幾頁有準則，再擷取那幾頁

In [2]:
pdf = pr.open(filename)
ps = pdf.pages
df_new=[]
new_page=[]
columns_count = []
 # 取得總頁數
total_pages = len(pdf.pages)
# 設定起始頁數
start_page = int(total_pages * 0.8)
    
# 迭代每頁，從設定的起始頁開始
for page in pdf.pages[start_page:]:
    text = page.extract_text()
    if ("GRI" and "頁" )in text:
        new_page.append(page)
        for page in new_page:
            tables = page.extract_tables()
            for df in tables:
                df = pd.DataFrame(df, columns=df[0])
                #df=df.drop_duplicates()
                df_new.append(df) 

In [3]:
UR=[]
final=[]
pattern="\d{3}[\-－]\d{1,2}"

for df in df_new:
    if df.shape[1] >= 4:
        final.append(df)
        for df in final:
            # 检查当前DataFrame对象的前两列是否与正则表达式匹配
            col1 = df.iloc[:, 0]
            col2 = df.iloc[:, 1]
            matches = col1.str.contains(pattern) | col2.str.contains(pattern)

            # 如果至少有一个单元格与正则表达式匹配，则将该DataFrame添加到UR列表中
            if matches.any():
                UR.append(df)
UR

[                       GRI\n準則                          揭露項目  \
 0                      GRI\n準則                          揭露項目   
 1                         一般揭露                          None   
 2          GRI 102 2016\n：一般揭露        102-53\n可回答報告書相關問題的聯絡人   
 3                         None        102-54 GRI\n依循 準則報導的宣告   
 4                         None              102-55 GRI\n內容索引   
 5                         None             102-56 /\n外部保證 確信   
 6                         重大主題                          None   
 7                         風險管理                          None   
 8       GRI 103\n：\n2016\n管理方針             103-1\n解釋重大主題與其邊界   
 9                         None             103-2\n管理方針與其組成部分   
 10                        None                103-3\n管理方針的評估   
 11         GRI 201 2016\n：經濟績效   201-2\n氣候變遷所產生的財務影響及其它風險與機會   
 12                   公司治理與誠信經營                          None   
 13      GRI 103\n：\n2016\n管理方針             103-1\n解釋重大主題與其邊界   
 14                      

## 輸出表格為.XLSX

In [12]:
excel_name=filename+".xlsx"
output_excel=merged_df.to_excel(excel_name, index=False)

## 讀取整理好之excel表格頁碼切割pdf並貼回路徑

In [25]:
# Read in the Excel file as a dataframe

import pandas as pd
import numpy as np
import openpyxl
import os
from openpyxl import load_workbook
from pdfrw import PdfReader, PdfWriter


df = pd.read_excel(excel_name, engine='openpyxl')
df

# Loop through each row in the DataFrame
for i in range(df.shape[0]):
    # Split the pages string into individual page numbers
    pages = list(map(int, df.loc[i, 'pages'].split(',')))
    
    # Generate the corresponding PDF file
    output_file = os.path.join("C:/Users/pojid/01/output", str(i+1) + '.pdf')
    writer = PdfWriter()
    for page in pages:
        writer.addpage(PdfReader(filename).pages[page-1])
    writer.write(output_file)
    
    # Update the pdf_path column in the DataFrame
    df.loc[i, 'pdf_path'] = output_file


KeyError: 'pages'

In [7]:
# create a Pandas Excel writer using OpenPyXL as the engine
writer = pd.ExcelWriter(output_excel, engine='openpyxl')

# write the DataFrame to the Excel file
df.to_excel(writer, index=False)

# save the changes to the file
writer.save()

C:\Users\pojid\AppData\Local\Temp\ipykernel_16164\2325829603.py:9: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
